# **遙控偵測車**

### 首先重設CSI Camera

In [1]:
!echo 'nano1111' | sudo -S systemctl restart nvargus-daemon && printf '\n'  # Reset CSI Camera

[sudo] password for nano: 


### 載入套件

In [3]:
from ipywidgets import widgets
from IPython.display import display

from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
from jetbot import Robot
from SRDevice import Device

### 建立物件

In [4]:
robot = Robot()
camera = CSICamera(width=384, height=216)
controller = widgets.Controller(index=0)
dev = Device()


### 建構監視畫面

In [11]:
image = camera.read()
current_image = widgets.Image(format='jpeg', width=384, height=216)
current_image.value = bgr8_to_jpeg(image)

snapshot_image = widgets.Image(format='jpeg', width=192, height=108)
snapshot_image.value = bgr8_to_jpeg(image)

lab_t = widgets.Label(value='溫度')
lab_h = widgets.Label(value='濕度')
lab_gas = widgets.Label(value='瓦斯')
lab_pir = widgets.Label(value='人體')

info_container = widgets.HBox([lab_t, lab_h, lab_gas, lab_pir])
control_container = widgets.VBox([current_image, info_container, snapshot_image])

display(control_container)
display(controller)

Controller(axes=(Axis(value=-0.003921568393707275), Axis(value=-0.003921568393707275), Axis(value=1.0), Axis(v…

### 連線

In [17]:
def update_image(change):
    image = change['new']
    current_image.value = bgr8_to_jpeg(image)
    
camera.running = True
camera.observe(update_image, names='value')

In [18]:
def update_turn(change):
    speed = change['new']/3
    robot.left_motor.value = speed
    robot.right_motor.value = -speed

def update_direction(change):
    speed = change['new']/3
    robot.left_motor.value = -speed
    robot.right_motor.value = -speed

controller.axes[0].observe(update_turn, names='value')    
controller.axes[1].observe(update_direction, names='value')

In [19]:
def update_info(change):
    info = change['new']
#     print(info)
    try:
        (type, t, h, g, p) = info.split(' ')
        if type=='DATA:':
            lab_t.value = '溫度: ' + t + '°C'
            lab_h.value = '濕度: ' + h + '%'
            lab_gas.value = '瓦斯: ' + ('有' if g == '0' else '無')
            lab_pir.value = '人體: ' + ('有' if p == '1' else '無')
    except:
        print('ERR')
        
dev.running = True
dev.observe(update_info, names='value')


In [20]:
def save_snapshot(change):
    import uuid
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(current_image.value)
            
        # display snapshot that was saved
        snapshot_image.value = current_image.value

controller.buttons[0].observe(save_snapshot, names='value')


### 離線

In [16]:
camera.unobserve(update_image, names='value')
camera.running = False

controller.axes[0].unobserve(update_turn, names='value')
controller.axes[1].unobserve(update_direction, names='value')
controller.buttons[0].unobserve(save_snapshot, names='value')

dev.unobserve(update_info, names='value')
dev.running = False